# Visualizing CASP Data

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [13]:
def make_sample(example):
    sample = []
    
    sample.append(example[1].rstrip())
    sample.append(example[3].rstrip())
    
    pssm = []
    for i in range(5, 26):
        pssm.append([float(x) for x in example[i].split()])
    sample.append(pssm)
    
    tertiary = []
    for i in range(27, 30):
        tertiary.append([float(x) for x in example[i].split()])
    sample.append(tertiary)
    
    sample.append(example[31].rstrip())
    
    return sample

In [14]:
with open('data/text_based/training/training_30') as raw_data:
    example, samples = [], []
    for line in raw_data:
        example.append(line)
        if len(example) == 33:
            samples.append(make_sample(example))
            example = []
print(len(samples))

22344
